# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
import functions_notebook_arinew as fan
from datetime import timedelta
from scipy.stats import entropy
import os


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### Estrazione features, creazione dataframe, scrittura dataframes su disco
levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']
# clean captures
clean_capture = 'captures\\captures1_v2\\captures1_v2\\clean\\eth2dump-clean-0,5_1.json'

# ping flooding captures 
pf_capture_1 = 'captures\\captures1_v2\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'
pf_capture_2 = 'captures\\captures2\\captures2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'
pf_capture_3='captures\\captures3\\captures3\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0.5h_1.json'

# nome dataframe su disco
dfclean_disk = "compressed_datasets\\captures1_v2\\clean\\df_clean.gzip"
dfpf1_disk = "compressed_datasets\\captures1_v2\\pingFloodDDoS\\dfpf1.gzip"
dfpf2_disk = "compressed_datasets\\captures2\\pingFloodDDoS\\dfpf2.gzipp"
dfpf3_disk = "compressed_datasets\\captures3\\pingFloodDDoS\\dfpf3.gzip"

if not os.path.isfile(dfclean_disk):
	packets_cc = fn.load_packets_from_file(clean_capture)
	features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)
	df_cc = pd.DataFrame(features_cc)
	features_cc.clear()
	df_cc.to_pickle(dfclean_disk, compression='gzip')
else:
	df_cc = pd.read_pickle(dfclean_disk, compression='gzip')

if not os.path.isfile(dfpf1_disk):
	pkt_pf_15m_05h = fn.load_packets_from_file(pf_capture_1)
	features_pf_15m_05h = fn.extract_features_from_packets(pkt_pf_15m_05h, levels_of_interest)
	df = pd.DataFrame(features_pf_15m_05h)
	features_pf_15m_05h.clear()
	df.to_pickle(dfpf1_disk, compression='gzip')
else:
	df = pd.read_pickle(dfpf1_disk, compression='gzip')

if not os.path.isfile(dfpf2_disk):
	pkt_pf_15m_05h_2 = fn.load_packets_from_file(pf_capture_2)
	features_pf_15m_05h_2 = fn.extract_features_from_packets(pkt_pf_15m_05h_2, levels_of_interest)
	df1 = pd.DataFrame(features_pf_15m_05h_2)
	features_pf_15m_05h_2.clear()
	df1.to_pickle(dfpf2_disk, compression='gzip')
else:
	df1 = pd.read_pickle(dfpf2_disk, compression='gzip')

if not os.path.isfile(dfpf3_disk):
	pkt_pf_15m_05h_3 = fn.load_packets_from_file(pf_capture_3)
	features_pf_15m_05h_3 = fn.extract_features_from_packets(pkt_pf_15m_05h_3, levels_of_interest)
	df2 = pd.DataFrame(features_pf_15m_05h_3)
	features_pf_15m_05h_3.clear()
	df2.to_pickle(dfpf3_disk, compression='gzip')
else:
	df2 = pd.read_pickle(dfpf3_disk, compression='gzip')

In [3]:
print(len(df_cc))
print(len(df))
print(len(df1))
print(len(df2))

35430
197778
212192
212241


In [4]:
fn.calcolo_features_binarie(df_cc)
fn.calcolo_features_binarie(df)
fn.calcolo_features_binarie(df1)
fn.calcolo_features_binarie(df2)

In [5]:
df_cc['frame.frame.time_utc'] = pd.to_datetime(df_cc['frame.frame.time_utc'])
df['frame.frame.time_utc'] = pd.to_datetime(df['frame.frame.time_utc'])
df1['frame.frame.time_utc'] = pd.to_datetime(df1['frame.frame.time_utc'])
df2['frame.frame.time_utc'] = pd.to_datetime(df2['frame.frame.time_utc'])

In [6]:
df_cc.sort_values(by='frame.frame.time_utc')
df.sort_values(by='frame.frame.time_utc')
df1.sort_values(by='frame.frame.time_utc')
df2.sort_values(by='frame.frame.time_utc')


,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,...,smb,dhcp,dhcpv6,icmpv6,icmp,igmp,ipv6.hopopts,browser,tcp,ip
0,1,"Aug 8, 2018 19:37:05.555487000 ora legale Eur...",2018-08-08 17:37:05.555487+00:00,1533749825.555487000,0.000000000,0.000000000,0.000000000,0.000000000,1,66,...,0,0,0,0,0,0,0,0,1,1
1,1,"Aug 8, 2018 19:37:05.562744000 ora legale Eur...",2018-08-08 17:37:05.562744+00:00,1533749825.562744000,0.000000000,0.007257000,0.007257000,0.007257000,2,85,...,0,0,0,0,0,0,0,0,1,1
2,1,"Aug 8, 2018 19:37:05.773296000 ora legale Eur...",2018-08-08 17:37:05.773296+00:00,1533749825.773296000,0.000000000,0.210552000,0.210552000,0.217809000,3,60,...,0,0,0,0,0,0,0,0,1,1
3,1,"Aug 8, 2018 19:37:05.867520000 ora legale Eur...",2018-08-08 17:37:05.867520+00:00,1533749825.867520000,0.000000000,0.094224000,0.094224000,0.312033000,4,66,...,0,0,0,0,0,0,0,0,1,1
4,1,"Aug 8, 2018 19:37:05.872108000 ora legale Eur...",2018-08-08 17:37:05.872108+00:00,1533749825.872108000,0.000000000,0.004588000,0.004588000,0.316621000,5,85,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212236,1,"Aug 8, 2018 20:07:04.798017000 ora legale Eur...",2018-08-08 18:07:04.798017+00:00,1533751624.798017000,0.000000000,0.094252000,0.094252000,1799.242530000,212237,66,...,0,0,0,0,0,0,0,0,1,1
212237,1,"Aug 8, 2018 20:07:04.807663000 ora legale Eur...",2018-08-08 18:07:04.807663+00:00,1533751624.807663000,0.000000000,0.009646000,0.009646000,1799.252176000,212238,85,...,0,0,0,0,0,0,0,0,1,1
212238,1,"Aug 8, 2018 20:07:05.015777000 ora legale Eur...",2018-08-08 18:07:05.015777+00:00,1533751625.015777000,0.000000000,0.208114000,0.208114000,1799.460290000,212239,60,...,0,0,0,0,0,0,0,0,1,1
212239,1,"Aug 8, 2018 20:07:05.110697000 ora legale Eur...",2018-08-08 18:07:05.110697+00:00,1533751625.110697000,0.000000000,0.094920000,0.094920000,1799.555210000,212240,66,...,0,0,0,0,0,0,0,0,1,1


In [7]:
start_time = df['frame.frame.time_utc'].iloc[0]
end_time=start_time+timedelta(seconds=298.8)
end_time2 = end_time + timedelta(seconds=900)

In [8]:
start_time_2 = df1['frame.frame.time_utc'].iloc[0]
end_time_2=start_time_2+timedelta(seconds=299.16)
end_time2_2 = end_time_2 + timedelta(seconds=900)

In [9]:
start_time_3 = df2['frame.frame.time_utc'].iloc[0]
end_time_3=start_time_3+timedelta(seconds=240.59)
end_time2_3 = end_time_3 + timedelta(seconds=959)

In [10]:
df_cc1 = df[(df['frame.frame.time_utc'] >= start_time) & (df['frame.frame.time_utc'] < end_time)].copy()
df_attack = df[(df['frame.frame.time_utc'] >= end_time) & (df['frame.frame.time_utc'] <= end_time2)].copy()
df_cc2 = df[(df['frame.frame.time_utc'] > end_time2)].copy()

In [11]:
df1_cc1_2 = df1[(df1['frame.frame.time_utc'] >= start_time_2) & (df1['frame.frame.time_utc'] < end_time_2)].copy()
df1_attack_2 = df1[(df1['frame.frame.time_utc'] >= end_time_2) & (df1['frame.frame.time_utc'] <= end_time2_2)].copy()
df1_cc2_2 = df1[(df1['frame.frame.time_utc'] > end_time2_2)].copy()

In [12]:
df2_cc1_3 = df2[(df2['frame.frame.time_utc'] >= start_time_3) & (df2['frame.frame.time_utc'] < end_time_3)].copy()
df2_attack_3 = df2[(df2['frame.frame.time_utc'] >= end_time_3) & (df2['frame.frame.time_utc'] <= end_time2_3)].copy()
df2_cc2_3 = df2[(df2['frame.frame.time_utc'] > end_time2_3)].copy()

In [13]:
df_cc_aggregation = fan.calcola_features(df_cc)
df_cc1_aggregation = fan.calcola_features(df_cc1)
df_cc2_aggregation = fan.calcola_features(df_cc2)
df_att_aggregation = fan.calcola_features(df_attack)

In [14]:
df1_cc1_aggregation = fan.calcola_features(df1_cc1_2)
df1_att_aggregation = fan.calcola_features(df1_attack_2)
df1_cc2_aggregation = fan.calcola_features(df1_cc2_2)

In [15]:
df2_cc1_aggregation = fan.calcola_features(df2_cc1_3)
df2_att_aggregation = fan.calcola_features(df2_attack_3)
df2_cc2_aggregation = fan.calcola_features(df2_cc2_3)

In [16]:
df_cc_aggregation['label']=0
df_cc1_aggregation['label']=0
df_cc2_aggregation ['label']=0
df_att_aggregation ['label']=1

df1_cc1_aggregation['label']=0
df1_att_aggregation['label']=1
df1_cc2_aggregation['label']=0

df2_cc1_aggregation['label']=0
df2_att_aggregation['label']=1
df2_cc2_aggregation['label']=0

In [17]:
df_tot = pd.concat([df_cc_aggregation,df_cc1_aggregation,df_cc2_aggregation,df_att_aggregation,df1_cc1_aggregation,df1_att_aggregation,df1_cc2_aggregation,df2_cc1_aggregation, df2_att_aggregation,df2_cc2_aggregation], axis=0)

In [18]:
df_tot['ipat_std'] = df_tot['ipat_std'].dt.total_seconds()
df_tot['ipat_max'] = df_tot['ipat_max'].dt.total_seconds()
df_tot['ipat_min'] = df_tot['ipat_min'].dt.total_seconds()
df_tot['ipat_mode'] = df_tot['ipat_mode'].dt.total_seconds()

In [19]:
df_tot = df_tot.fillna(0)

In [20]:
X = df_tot.drop(columns='label')  
y = df_tot['label'] 

In [21]:
from sklearn.feature_selection import f_classif
f_values, p_values = f_classif(X, y)
anova_results = pd.DataFrame({
    'Feature': X.columns,
    'F-value': f_values,
    'P-value': p_values
})
anova_results_sorted = anova_results.sort_values(by='P-value')


# Aggiungi una colonna per segnalare se la feature può essere scartata (P-value > 0.05)
anova_results_sorted['Discard'] = anova_results_sorted['P-value'] > 0.05



# Mostra le colonne che possono essere scartate
features_to_discard = anova_results_sorted[anova_results_sorted['Discard']]['Feature'].tolist()
print("\nColonne che possono essere scartate:", features_to_discard)


Colonne che possono essere scartate: ['udp_count', 'ipat_min', 'tcp_syn_count', 'ip_dst_mode', 'ip_src_mode']


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 8 51 52] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [22]:
df_tot=df_tot.drop(columns=['udp_count', 'ipat_min', 'tcp_syn_count', 'ip_dst_mode', 'ip_src_mode'])

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt
correlation_matrix = df_tot.corr(method='pearson')
correlation_matrix
correlation_matrix1 = correlation_matrix.where(correlation_matrix >0.80)

correlation_matrix1


,eth_count,ip_count,arp_count,tcp_count,mbtcp_count,icmp_count,pkt_count,per_eth_count,per_ip_count,per_icmp_count,...,entropy_iplen,bytes_per_timeunit,pkt_per_timeunit,modbus_response_count,modbus_request_count,modb_req_resp_fraction,icmp_request_count,icmp_response_count,icmp_req_resp_fraction,label
eth_count,1.000000,0.999987,NaN,NaN,NaN,0.998765,1.000000,NaN,0.850273,0.993932,...,NaN,0.998965,1.000000,NaN,NaN,NaN,0.966788,NaN,NaN,0.976738
ip_count,0.999987,1.000000,NaN,NaN,NaN,0.998849,0.999987,NaN,0.851390,0.994051,...,NaN,0.998894,0.999987,NaN,NaN,NaN,0.966855,NaN,NaN,0.976847
arp_count,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tcp_count,NaN,NaN,NaN,1.000000,0.942868,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.931662,0.931988,0.908390,NaN,NaN,NaN,NaN
mbtcp_count,NaN,NaN,NaN,0.942868,1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.998833,0.998669,0.987451,NaN,NaN,NaN,NaN
icmp_count,0.998765,0.998849,NaN,NaN,NaN,1.000000,0.998765,NaN,0.840212,0.996294,...,NaN,0.995597,0.998765,NaN,NaN,NaN,0.968180,NaN,NaN,0.977865
pkt_count,1.000000,0.999987,NaN,NaN,NaN,0.998765,1.000000,NaN,0.850273,0.993932,...,NaN,0.998965,1.000000,NaN,NaN,NaN,0.966788,NaN,NaN,0.976738
per_eth_count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
per_ip_count,0.850273,0.851390,NaN,NaN,NaN,0.840212,0.850273,NaN,1.000000,0.842065,...,NaN,0.859232,0.850273,NaN,NaN,NaN,0.818003,NaN,NaN,0.832062
per_icmp_count,0.993932,0.994051,NaN,NaN,NaN,0.996294,0.993932,NaN,0.842065,1.000000,...,NaN,0.989824,0.993932,NaN,NaN,NaN,0.965753,NaN,NaN,0.978861


In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score


In [25]:
#DecisionTree
random_statev=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_statev)
clf1 = DecisionTreeClassifier(random_state=random_statev)
parametri = {
    'criterion': ['gini', 'entropy'],        
    'max_depth': [10, 20, 30],          
    'min_samples_split': [2, 10, 20],       
    'min_samples_leaf': [1, 5, 10],           
    'max_features': [None,'sqrt', 'log2'],   
    'splitter': ['best', 'random']  
}          
grid_search = GridSearchCV(estimator=clf1, param_grid=parametri, 
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Migliori parametri trovati dalla GridSearch:", grid_search.best_params_)
best_dt = grid_search.best_estimator_
accuracy = best_dt.score(X_test, y_test)
print(f"Accuratezza sul test set: {accuracy:.4f}")


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Migliori parametri trovati dalla GridSearch: {'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'}
Accuratezza sul test set: 0.9954


In [26]:
#RandomForest
rf_clf2 = RandomForestClassifier(random_state=random_statev)
parametri = {
    'n_estimators': [100, 200, 500],          
    'max_depth': [None, 10, 20, 30],         
    'min_samples_split': [2, 5, 10],          
    'min_samples_leaf': [1, 2, 4],           
    'max_features': ['sqrt', 'log2', None],  
    'bootstrap': [True, False]                
}
grid_search_rf = GridSearchCV(estimator=rf_clf2, param_grid=parametri, 
                              cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

grid_search_rf.fit(X_train, y_train)
print("Migliori parametri trovati dalla GridSearch:", grid_search_rf.best_params_)
best_rf = grid_search_rf.best_estimator_
accuracy = best_rf.score(X_test, y_test)
print(f"Accuratezza sul test set: {accuracy:.4f}")

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Migliori parametri trovati dalla GridSearch: {'bootstrap': True, 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuratezza sul test set: 0.9954


In [27]:
#knn
knn = KNeighborsClassifier()
parametri = {
    'n_neighbors': [3, 5, 7, 10],              
    'weights': ['uniform', 'distance'],        
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  
    'leaf_size': [30, 40, 50],                 
    'metric': ['euclidean', 'manhattan']       
}
grid_search_knn = GridSearchCV(estimator=knn, param_grid=parametri, 
                                cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search_knn.fit(X_train, y_train)
print("Migliori parametri trovati dalla GridSearch:", grid_search_knn.best_params_)
best_knn = grid_search_knn.best_estimator_
accuracy = best_knn.score(X_test, y_test)
print(f"Accuratezza sul test set: {accuracy:.4f}")


Fitting 5 folds for each of 192 candidates, totalling 960 fits


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Migliori parametri trovati dalla GridSearch: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
Accuratezza sul test set: 0.9885


In [31]:


# Normalizzazione dei dati
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Inizializziamo il SVC
svm = SVC(random_state=42)

# Definiamo la griglia di parametri da testare
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'kernel': ['linear', 'rbf', 'poly']
}

# Creiamo l'oggetto GridSearchCV
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, 
                           scoring='f1_weighted', cv=5, verbose=1, n_jobs=-1)

# Alleniamo il modello utilizzando Grid Search
grid_search.fit(X_train, y_train)

# Stampa i migliori parametri trovati
print("Best Parameters:", grid_search.best_params_)

# Utilizziamo il miglior modello trovato
best_svm = grid_search.best_estimator_

# Prediciamo sui dati di test
y_pred = best_svm.predict(X_test)

# Calcoliamo l'accuratezza e l'F1 score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Stampiamo i risultati
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"F1 Score: {f1:.5f}")



Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Parameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 99.54%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       269
           1       0.99      1.00      0.99       166

    accuracy                           1.00       435
   macro avg       0.99      1.00      1.00       435
weighted avg       1.00      1.00      1.00       435

F1 Score: 0.99541
